- As project 2, I made functions at .py file, and then ,plot results in .ipynb file. 
- I condider Repeated First Price Auction(hereinafter called repeated FPA), not second price auction.
- Players in Repeated FPA must have algorithm whose regret coverges to 0. 

# Project 3: Repeated First Price Auction

## Parameters

- n_rounds: 1000 - number of rounds per simulation
- k: 100 - number of discrete arms (discretization level)
- n_mc: 100 - number of Monte Carlo simulation runs
- h: scaling parameter (default: value) - used in Exponential Weight algorithms
- value (v): 10.0 - player's value for the item (default)
- learning_rate: sqrt(log(k) / n) - learning rate for Exponential Weight algorithms (default for flexible)
- delta: 0.95 - discount factor for long-term algorithm (default)
- observation_rounds: 20 - number of observation rounds for exploitation algorithm (default)

## Algorithms

1. 1_myopic: Myopic algorithm - maximizes current round expected utility
2. 2_long: Long-term algorithm - maximizes discounted long-term utility
3. 3_flexible: Flexible algorithm - Exponential Weight with learning_rate = np.sqrt(np.log(k) / n)
4. 4_cool: Cool algorithm - always bids v/2
5. 5_feeling: Feeling algorithm - Exponential Weight over 5 strategy arms
6. 6_Exploitation: Exploitation algorithm - waits and exploits when opponent bids low

In [1]:
# import library
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import statistics

In [ ]:
# I implemented both full and partial information version. 
# In full information version, I immediately get Nash Equilibrium. a player knows everything including other players' bids and utilities.
# In partial information version, a player only knows his bids and his utility, not other players' bids and utilities, which requres need to consider more  complex strategy. 

# Here is our model(fpa.py).
# At each round i, each player decide their bid based on the past information and his/her strategy.
# Then, FPA is played.
# After FPA, each player get utility.
# Then, this process is repeated.

# As is often the case, notation is followed.
# n is the number of players.
# v is the value of the item.
# b is the bid of the player.
# u is the utility of the player.
# p is the probability of winning.

# regret was determined by the difference between the utility he could get (best fixed action in hindsight) and the utility he got.

# Here is our Assumption.
# Partial information
# value is determined by the nature, will not change across rounds.
# If win, utility is calculated by value - bid, if lose, utility is 0.
# If there is a tie in bid, then the players who bid highest get the item by flipping a coin.
# discrete value and discrete bid to calculate. 

# Monte carlo simulation to get the result.
# 10000 rounds, 2000 times.

# I proposed 7 types of algorithms to consider.
# I named them as Myopic Model, Long Model, Optimal Model, Uniform Model, FTL Model, Cool Model, and Feeling Model.
# Each algorithm can be defined by a function whose argument is the past information and the number of rounds, and returns the bid.

# First, I made myopic algorithm which is tuned to cares about only current round. (1_myopic Model)
    # This can be implemented by using our Project1's idea!! culculate other bidder's CDF of bid. 
    # At each round, it calculate the CDF of other bidders' highest bid, then calculate the probability of winning.
    # Hence, we get expected value of bid.

# Second, I made algorithm which is tuned to cares about long-term payoff. (2_long Model)
    # This is really difficult to implement. 
    # At each round, it calculate the CDF of other bidders' highest bid.
    # I used time-discounted sum, and algorithm will make long-term payoff.
    # This algorithm represent the smart player in auction.

# Third, I made algorithm which is tuned optimally to balance between two algorithms above. (3_flexible Model)
    # This is basic EXponential weight algorithm with nice learning rate = sqrt(log(k) / n).

# Forth, I made algorithm which bid random value between 0 and v. (4_uniform Model)
    # I concerned that this algorithm's regret do not converge to 0. 

# Fifth, I made algorithm which is tuned to be FTL. (5_ftl Model)
    # This is basic EXponential weight algorithm with nice learning rate = 100

# Sixth, I made algorithm which always bids theoretical optimal value. (4_cool Model)
    # Always pay n-1/n * v. 

 # Seventh, I made algorithm which is tuned to choose algorithm based on the past rounds results. (5_feeling Model)
    # I model the basic people's feeling of bidding. If you lose many times, you tend to bid higher.
    # what is the difference between this and 3_flexible Model is that 6th choose which algorithm to use. 
    # For example, 
    # first strategy is to bid aggressively 3/4 of v
    # second strategy is to bid normaly 1/2 of v
    # third strategy is to bid conservatively 1/4 of v
    # if you lose many times, you tend to use first strategy.
    # if you win many times, you tend to use second strategy.
    # if you win and lose many times, you tend to use third strategy.

In [ ]:
# collect algorithm
import importlib
import sys
from pathlib import Path

# Add algorithm directory to path (works in Jupyter notebooks)
algorithm_dir = Path('algorithm')
if str(algorithm_dir.resolve()) not in sys.path:
    sys.path.insert(0, str(algorithm_dir.resolve()))

# Import modules with numeric names using importlib and create aliases
myopic = importlib.import_module('1_myopic')
long = importlib.import_module('2_long')
flexible = importlib.import_module('3_flexible')
# Note: 4_random and 5_ftl are removed - they can be represented by flexible with different learning rates
# 4_random: random strategy (can be removed as it doesn't converge to 0 regret)
# 5_ftl: flexible with learning_rate = 100
cool = importlib.import_module('4_cool')
feeling = importlib.import_module('5_feeling')
exploitation = importlib.import_module('6_Exploitation')

# Import other modules
import two_repeatedFPA

# Part 1

In [ ]:
# we simulate the game with mixed players who use above algorithms(This is Part1). 

# Part 1: Outcomes from No-regret Learning in Games
# Experiment design:
# - Different combinations of algorithms playing against each other
# - For example: Myopic vs Long, Flexible vs FTL, etc.
# - Also consider: same algorithm vs different algorithm matchups
# - Number of players: n = 3 (or other fixed value)
# - Value: v = 10 (or other fixed/discrete values)

# Part 1 Questions to Answer:
# 1. Do the algorithms converge to a Nash equilibrium?
#    - Track bid distributions over time
#    - Check if they stabilize to Nash equilibrium bids
# 2. In games with multiple Nash equilibria, which one do they converge to?
#    - First Price Auction may have multiple equilibria depending on value distributions
#    - Need to identify which equilibrium is reached
# 3. Convergence rate analysis
#    - How many rounds until convergence?

# Learning rate analysis:
# - For 3_flexible (learning rate = sqrt(log(k) / n))
# - For 5_ftl (learning rate = 100)
# - Experiment with different learning rates
# - Meta-game: What if players choose learning rates strategically?
#   → Nash equilibrium in the meta-game where players choose learning rates

# Comparison metrics:
# - Average regret over time (should converge to 0)
# - Final average utility/payoff
# - Convergence speed
# - Stability of bids (variance over last N rounds)

# Comparison metrics:
# - Average regret over time (should converge to 0)
# - Final average utility/payoff
# - Convergence speed
# - Stability of bids (variance over last N rounds)


# まず, EX3のために ,b round, k arm 離散化の度合い, h : scaling parameterを設定したい
# n = 1000, 
# k = 100
# h は各々のvalue.

#　regret, utility, 勝率の変化を毎回のMCの際に記録しておく必要がある.

In [ ]:
# Part 1 Implementation
# Parameters
n_rounds = 1000  # number of rounds
k = 100  # number of arms (discretization)
n_mc = 100  # number of Monte Carlo simulations

# Import simulation and plotting functions from two_repeatedFPA.py
from two_repeatedFPA import run_repeated_fpa, plot_part1_results

print("Part 1 simulation functions imported from two_repeatedFPA.py")
    """
    Run repeated First Price Auction simulation
    
    Args:
        player1_config: tuple of (algorithm_function, value, env_state_dict)
        player2_config: tuple of (algorithm_function, value, env_state_dict)
        n_rounds: number of rounds per simulation
        n_mc: number of Monte Carlo runs
    
    Returns:
        results: dict with regret, utility, win_rate for each player
    """
    alg1_func, v1, env1 = player1_config
    alg2_func, v2, env2 = player2_config
    
    # Initialize environment state for each player
    env1.setdefault('k', k)
    env1.setdefault('h', v1)
    env2.setdefault('k', k)
    env2.setdefault('h', v2)
    
    # Storage for all MC runs
    all_regret1 = []
    all_regret2 = []
    all_utility1 = []
    all_utility2 = []
    all_win_rate1 = []
    all_win_rate2 = []
    all_bids1 = []
    all_bids2 = []
    all_regret_history1 = []
    all_regret_history2 = []
    
    # Monte Carlo loop
    for mc_iter in range(n_mc):
        # Reset for each MC run
        history1 = []
        history2 = []
        env1_state = env1.copy()
        env2_state = env2.copy()
        env1_state['cumulative_payoffs'] = np.zeros(k)
        env2_state['cumulative_payoffs'] = np.zeros(k)
        
        total_utility1 = 0
        total_utility2 = 0
        wins1 = 0
        wins2 = 0
        
        bids1_history = []
        bids2_history = []
        regret_history1 = []
        regret_history2 = []
        opponent_bids1 = []
        opponent_bids2 = []
        
        # Round loop
        for round_num in range(n_rounds):
            # Get bids from each player
            bid1 = alg1_func(0, v1, round_num, history1, env1_state)
            bid2 = alg2_func(1, v2, round_num, history2, env2_state)
            
            bids1_history.append(bid1)
            bids2_history.append(bid2)
            
            # Play FPA
            auction = fpa.FPA()
            alloc1, alloc2 = auction.play_round(bid1, bid2)
            
            # Calculate utilities
            utility1 = utility.calculate_utility(v1, alloc1, bid1)
            utility2 = utility.calculate_utility(v2, alloc2, bid2)
            
            # Update totals
            total_utility1 += utility1
            total_utility2 += utility2
            
            # Track wins
            if alloc1 > 0.5:
                wins1 += 1
            elif alloc2 > 0.5:
                wins2 += 1
            else:  # tie
                if np.random.random() < 0.5:
                    wins1 += 1
                else:
                    wins2 += 1
            
            # Update history
            won1 = (alloc1 > 0.5) or (alloc1 == 0.5 and np.random.random() < 0.5)
            won2 = (alloc2 > 0.5) or (alloc2 == 0.5 and np.random.random() < 0.5)
            history1.append((bid1, utility1, won1))
            history2.append((bid2, utility2, won2))
            
            # Store opponent bids for regret calculation
            opponent_bids1.append(bid2)
            opponent_bids2.append(bid1)
            
            # Calculate regret: best fixed action in hindsight
            bid_grid = np.linspace(0, max(v1, v2), k)
            
            best_fixed_utility1 = 0
            for fixed_bid in bid_grid:
                if fixed_bid > v1:
                    continue
                fixed_utility1 = sum([
                    utility.calculate_utility(v1,
                        fpa.FPA().play_round(fixed_bid, opp_bid)[0],
                        fixed_bid)
                    for opp_bid in opponent_bids1
                ])
                best_fixed_utility1 = max(best_fixed_utility1, fixed_utility1)
            
            best_fixed_utility2 = 0
            for fixed_bid in bid_grid:
                if fixed_bid > v2:
                    continue
                fixed_utility2 = sum([
                    utility.calculate_utility(v2,
                        fpa.FPA().play_round(opp_bid, fixed_bid)[1],
                        fixed_bid)
                    for opp_bid in opponent_bids2
                ])
                best_fixed_utility2 = max(best_fixed_utility2, fixed_utility2)
            
            # Calculate regret at this round
            regret1_round = best_fixed_utility1 - total_utility1
            regret2_round = best_fixed_utility2 - total_utility2
            regret_history1.append(regret1_round)
            regret_history2.append(regret2_round)
        
        all_regret1.append(regret_history1[-1])
        all_regret2.append(regret_history2[-1])
        all_utility1.append(total_utility1)
        all_utility2.append(total_utility2)
        all_win_rate1.append(wins1 / n_rounds)
        all_win_rate2.append(wins2 / n_rounds)
        all_bids1.append(bids1_history)
        all_bids2.append(bids2_history)
        all_regret_history1.append(regret_history1)
        all_regret_history2.append(regret_history2)
        
        if (mc_iter + 1) % 10 == 0:
            print(f"MC iteration {mc_iter + 1}/{n_mc} completed")
    
    return {
        'regret1': np.array(all_regret1),
        'regret2': np.array(all_regret2),
        'utility1': np.array(all_utility1),
        'utility2': np.array(all_utility2),
        'win_rate1': np.array(all_win_rate1),
        'win_rate2': np.array(all_win_rate2),
        'bids1': all_bids1,
        'bids2': all_bids2,
        'regret_history1': all_regret_history1,
        'regret_history2': all_regret_history2
    }


Part 1 simulation functions defined


In [ ]:
# Plotting functions are now in two_repeatedFPA.py
# Use plot_part1_results() from two_repeatedFPA module


Plotting functions defined


In [ ]:
# Run simulation: Myopic vs Flexible
print("Running: Myopic vs Flexible")
v1, v2 = 10.0, 10.0
player1 = (myopic.myopic_algorithm, v1, {'k': k, 'h': v1})
player2 = (flexible.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': np.sqrt(np.log(k) / n_rounds)})
results_myopic_vs_flexible = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
print("Completed: Myopic vs Flexible")
plot_part1_results(results_myopic_vs_flexible, title="Myopic vs Flexible")


Running: Myopic vs Flexible
MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed


In [ ]:
# Example: Run simulation with different algorithm combinations
# Test different matchups

# Note: n_mc = 100 can take a long time. For quick testing, use n_mc = 10
# Uncomment the examples below to run simulations

# Example 1: Myopic vs Flexible
# print("Running: Myopic vs Flexible")
# v1, v2 = 10.0, 10.0
# player1 = (myopic.myopic_algorithm, v1, {'k': k, 'h': v1})
# player2 = (flexible.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': np.sqrt(np.log(k) / n_rounds)})
# results_myopic_vs_flexible = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
# print("Completed: Myopic vs Flexible")
# plot_part1_results(results_myopic_vs_flexible, title="Myopic vs Flexible")

# Example 2: Flexible vs Exploitation
# print("Running: Flexible vs Exploitation")
# v1, v2 = 10.0, 10.0
# player1 = (flexible.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': np.sqrt(np.log(k) / n_rounds)})
# player2 = (exploitation.exploitation_algorithm, v2, {'k': k, 'h': v2, 'observation_rounds': 20})
# results_flexible_vs_exploitation = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
# print("Completed: Flexible vs Exploitation")
# plot_part1_results(results_flexible_vs_exploitation, title="Flexible vs Exploitation")


# Part 2

In [ ]:
# Part2
# - Exploitative strategy vs 1_myopic (Myopic strategy)
# - Exploitative strategy vs 2_long (Long-term strategy)
# - Exploitative strategy vs 3_flexible (learning rate = sqrt(log(k) / n))
# - Exploitative strategy vs 4_random (Random strategy)
# - Exploitative strategy vs 5_ftl (learning rate = 100)
# - Exploitative strategy vs 4_cool (Cool strategy)
# - Exploitative strategy vs 5_feeling (Feeling strategy)

In [ ]:
# Candidate of Exploitative strategy for each strategy(1,2,3,4,5,6,7)
# Strategy Against 1_myopic : 
# Strategy Against 2_long : 
# Strategy Against 3_flexible : 
# Strategy Against 4_random : 
# Strategy Against 5_ftl : 
# Strategy Against 4_cool : 
# Strategy Against 5_feeling : 